# 1. Import and Install Dependencies

In [1]:
# !pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [2]:
#test

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [4]:
import importlib # for reloading .py files
# WARNING: updates of config may require restart of kernel if reload is unsuccessful

import config
# reload config without restarting the kernel
importlib.reload(config)
from config import actions, no_sequences, sequence_length, DATA_PATH

import utils
# reload utils without restarting the kernel
importlib.reload(utils)
from utils import mediapipe_detection, extract_keypoints, draw_styled_landmarks, prob_viz

# 2. Keypoints using MP Holistic

In [5]:
from utils import mp_holistic, mp_drawing, set_camera_settings, default_fps

# 4. Setup Folders for Collection

In [6]:
# create folders



In [7]:
data_identifier = '1'

DATA_PATH = os.path.join(f"{data_identifier}_data", 'npy')

for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

VIDEO_PATH = os.path.join(f"{data_identifier}_data", 'videos')

for action in actions:

    try:
        os.makedirs(os.path.join(VIDEO_PATH, action))
    except:
        pass

# 5. Collect Keypoint Values - AND VIDEOS - for Training and Testing

In [8]:
actions = ['Hi', 'Yes', 'No', 'ThankYou', 'ILoveYou']

In [9]:
actions = [actions[0]]

In [10]:

cap = cv2.VideoCapture(1)  # Adjust device index as needed
actual_res, actual_fps = set_camera_settings(cap)

Requested Resolution: 1280x720, Actual Resolution: 1280x720
Requested FPS: 30, Actual FPS: 30.0


In [12]:
import time

frame_count = 0
fps_list = []
start_time = time.time()


# Set mediapipe model

fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter('output.avi', fourcc, 10.0, (int(cap.get(3)), int(cap.get(4))))


with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # Flag variable to signal breaking out of all loops
    break_all_loops = False

    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # fourcc = cv2.VideoWriter_fourcc(*'XVID')
            video_file_name = f'{VIDEO_PATH}/{action}/{sequence}.avi'
            out = cv2.VideoWriter(video_file_name, fourcc, 10.0, (int(cap.get(3)), int(cap.get(4))))


            for frame_num in range(sequence_length):
                if break_all_loops:
                    break

                ret, frame = cap.read()
                if not ret:
                    print("Failed to capture frame")

                    break  # If no frame is read, break out of the loop
                



                image, results = mediapipe_detection(frame, holistic)

                # write in the video file
                out.write(image)


                draw_styled_landmarks(image, results)

                if frame_num == 0:
                    for countdown in range(2, 0, -1):
                        cv2.putText(image, f'GET READY FOR "{action}" in {countdown}', (30,100),
                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0, 0), 4, cv2.LINE_AA)
                        cv2.imshow('OpenCV Feed', image)
                        cv2.waitKey(1000)
                        image = frame.copy()  # Reset image to clean frame
                else:
                    cv2.putText(image, f'RECORDING [{sequence + 1}/{no_sequences}] "{action}" - Frame {frame_num + 1}/{sequence_length}', (30,100),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0, 255), 4, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                frame_count += 1

                if frame_count % 100 == 0:
                    end_time = time.time()
                    elapsed = end_time - start_time
                    actual_fps = frame_count / elapsed
                    fps_list.append(actual_fps)
                    print("Actual FPS:", actual_fps)
                    # Reset timer and counter
                    start_time = time.time()
                    frame_count = 0

                if cv2.waitKey(10) & 0xFF == ord('t'):
                    break
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break_all_loops = True
                    break

            out.release()  # This should be here, inside the sequence loop but outside the frame loop

        if break_all_loops:
            break

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)


I0000 00:00:1715623367.420360       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M1


Actual FPS: 5.587033586774103
Actual FPS: 6.45455167723696
Actual FPS: 7.1103987741028485
Actual FPS: 5.944194324842463
Actual FPS: 7.029078500636292
Actual FPS: 6.790471926096915
Actual FPS: 6.100191894869519
Actual FPS: 6.565580748645489
Actual FPS: 6.917253870478812


In [13]:
sum(fps_list) / len(fps_list)

6.499861700409266

In [13]:
# fps_list
# [6.292873280920329,
#  6.651729536047263,
#  5.727258835325755,
#  11.471795841559342,
#  9.863759697661227]

[6.292873280920329,
 6.651729536047263,
 5.727258835325755,
 11.471795841559342,
 9.863759697661227]

In [ ]:
cap.release()
cv2.destroyAllWindows()